In [3]:
!pip install quandl

In [5]:
import pandas as pd
import quandl
from statsmodels.tsa.seasonal import seasonal_decompose

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [43]:

import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn')
# plt.style.use('seaborn-colorblind') #alternative
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
QUANDL_KEY = 'WrWh4k_6h13txBHprnGU' # replace {key} with your own API key  
quandl.ApiConfig.api_key = QUANDL_KEY

df = quandl.get(dataset='WGC/GOLD_MONAVG_USD',
                start_date='2000-01-01', 
                end_date='2011-12-31')

df.rename(columns={'Value': 'price'}, inplace=True)
df.head()
# df = df.resample('M').last()

,price
Date,
2000-01-31,284.31
2000-02-29,299.86
2000-03-31,286.39
2000-04-28,279.69
2000-04-30,279.69


In [44]:
WINDOW_SIZE = 12
df['rolling_mean'] = df.price.rolling(window=WINDOW_SIZE).mean()
df['rolling_std'] = df.price.rolling(window=WINDOW_SIZE).std()
df.plot(title='Gold Price')

plt.tight_layout()
#plt.savefig('images/ch3_im1.png')
plt.show()

AttributeError: ignored

In [1]:
from fbprophet import Prophet
import seabron as sns
import quandlimport pandas as pd


In [38]:
df = quandl.get(dataset='WGC/GOLD_MONAVG_USD',
                start_date='2000-01-01', 
                end_date='2005-12-31')

In [35]:
df['Value']

Date
2000-01-31    284.31
2000-02-29    299.86
2000-03-31    286.39
2000-04-28    279.69
2000-04-30    279.69
               ...  
2005-09-30    456.05
2005-10-31    469.90
2005-11-30    476.67
2005-12-30    510.10
2005-12-31    510.10
Name: Value, Length: 92, dtype: float64

In [34]:
df.loc['2000-01-31']

Value    284.31
Name: 2000-01-31 00:00:00, dtype: float64

In [39]:
df.reset_index(drop=False,inplace=True)

(92, 2)

In [41]:
df['Date']

0    2000-01-31
1    2000-02-29
2    2000-03-31
3    2000-04-28
4    2000-04-30
        ...    
87   2005-09-30
88   2005-10-31
89   2005-11-30
90   2005-12-30
91   2005-12-31
Name: Date, Length: 92, dtype: datetime64[ns]

In [45]:
df.rename(columns={'Date': 'ds', 'Value': 'y'}, inplace=True)

In [46]:
df.head()

,ds,y
0,2000-01-31,284.31
1,2000-02-29,299.86
2,2000-03-31,286.39
3,2000-04-28,279.69
4,2000-04-30,279.69


In [48]:
train_indices = df.ds.apply(lambda x: x.year).values < 2005
train_indices.shape


(92,)

In [49]:
df.loc[train_indices]

,ds,y
0,2000-01-31,284.31
1,2000-02-29,299.86
2,2000-03-31,286.39
3,2000-04-28,279.69
4,2000-04-30,279.69
...,...,...
72,2004-09-30,405.27
73,2004-10-29,420.46
74,2004-10-31,420.46
75,2004-11-30,439.37


In [50]:
df_train = df.loc[train_indices].dropna()
df_test = df.loc[~train_indices].reset_index(drop=True)

In [51]:
df_train

,ds,y
0,2000-01-31,284.31
1,2000-02-29,299.86
2,2000-03-31,286.39
3,2000-04-28,279.69
4,2000-04-30,279.69
...,...,...
72,2004-09-30,405.27
73,2004-10-29,420.46
74,2004-10-31,420.46
75,2004-11-30,439.37


In [52]:
model_prophet = Prophet(seasonality_mode='additive')
model_prophet.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model_prophet.fit(df_train)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
